In [1]:
!nvidia-smi

Fri May  5 12:43:22 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 6000                 On | 00000000:1B:00.0 Off |                  Off |
| 33%   30C    P8                9W / 260W|      1MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from time import time
from Loader import train_data_vgg16

2023-05-05 12:43:23.403061: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.operation_timeout_in_ms=60000
sess = tf.compat.v1.Session(config=config)

2023-05-05 12:43:24.766129: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 12:43:24.773024: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-05 12:43:24.774096: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-05 12:43:26.011233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-05 12:43:26.011744: I tensorflow/core/common_runtime/gpu/gpu_devic

In [4]:
####################################################################
# Following code releases unused GPU memory for others to use.
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
####################################################################

2023-05-05 12:43:29.818764: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-05 12:43:29.819329: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-05 12:43:29.819768: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:1c:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-05 12:43:29.820195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:1d:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-05 12:43:29.8

In [5]:
df = train_data_vgg16()
df.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,class
0,27.489744,0.0,0.000000,0.000000,0.000000,4.406033,0.000000,0.000000,0.0,9.508491,...,0.000000,0.0,0.0,0.0,3.239264,5.528229,0.0,0.0,0.00000,Real
1,0.000000,0.0,0.000000,27.298634,0.000000,21.718521,17.454420,0.000000,0.0,14.870244,...,0.000000,0.0,0.0,0.0,0.000000,22.730530,0.0,0.0,0.00000,Real
2,0.000000,0.0,0.000000,0.000000,0.000000,7.493243,0.000000,1.344108,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,Real
3,0.000000,0.0,29.934219,39.067970,0.000000,24.231049,6.911931,0.000000,0.0,0.000000,...,16.403683,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,17.57489,Real
4,0.000000,0.0,0.000000,24.634619,26.340034,0.000000,0.000000,0.000000,0.0,0.000000,...,13.876423,0.0,0.0,0.0,0.000000,28.267170,0.0,0.0,0.00000,Real


In [6]:
df_sample = df.sample(frac=1.0, random_state=0)

In [7]:
X = df_sample.drop('class', axis=1)
y = df_sample['class']
X.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
3582,0.000000,0.0,0.0,0.000000,4.260871,18.161793,0.0,0.000000,0.0,20.675310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000
10498,32.219925,0.0,0.0,24.435274,0.000000,0.000000,0.0,2.313794,0.0,10.794803,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000
3227,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,65.122879
21333,22.872917,0.0,0.0,0.000000,30.738342,34.451023,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000
3885,28.040337,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,65.730591,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50366,0.000000


In [ ]:
#Number of trees in the forest. We'll check 10, 20, 30, ... , 200
B = np.arange(20, 500, 20)
grid = {'n_estimators': B}

rf = RandomForestClassifier()
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    time_start = time()
    rfCV = GridSearchCV(rf, param_grid=grid, return_train_score=True, n_jobs = -1, verbose=2)
    rfCV.fit(X, y)
    time_stop = time()
    print('Elapsed Time for Logistic Regression:', time_stop-time_start)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2023-05-05 12:43:34.773984: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-05 12:43:34.774670: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-05 12:43:34.775129: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:1c:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-05 12:43:34.775546: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:1d:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-05 12:43:34.7

Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [ ]:
rfCV.best_score_

In [ ]:
rfCV.best_params_

In [ ]:
results = pd.DataFrame()
results['trees'] = grid['n_estimators']
results['train error'] = 1 - rfCV.cv_results_['mean_train_score']
results['validation error'] = 1 - rfCV.cv_results_['mean_test_score']

results.plot.line(x = 'trees', y = ['train error', 'validation error'])

In [ ]:
results = pd.DataFrame()
results['trees'] = grid['n_estimators']
results['train score'] = rfCV.cv_results_['mean_train_score']
results['validation score'] = rfCV.cv_results_['mean_test_score']

results.plot.line(x = 'trees', y = ['train score', 'validation score'])